In [1]:
import subprocess
import csv
import json
import os
import sys

# Inicializa acumuladores para o resumo
total_timeslots = 0
total_eprs_created = 0
total_eprs_consumed = 0
total_fidelity = 0.0
fidelity_count = 0
success_simulations = 0
success_attempts = []
error_simulations = 0
error_attempts = []

# Caminho do arquivo CSV
csv_file_path = "link_metrics_output.csv"

# Remove o arquivo CSV anterior se existir para evitar acumulação de dados
if os.path.exists(csv_file_path):
    os.remove(csv_file_path)

# Loop para rodar a simulação 150 vezes
simulation_index = 1  # Índice para manter a contagem linear das simulações bem-sucedidas ou falhas
for attempt in range(215):
    try:
        # Executa o script script.py em um novo processo
        result = subprocess.run(
            ["python", "script.py", str(attempt + 1)],
            capture_output=True,
            text=True,
            check=True
        )

        # Captura apenas a parte JSON da saída
        stdout = result.stdout.strip()
        start_index = stdout.find("{")  # Localiza o início do JSON
        end_index = stdout.rfind("}")  # Localiza o final do JSON

        if start_index == -1 or end_index == -1:
            print(f"Tentativa {attempt + 1}: Nenhum JSON encontrado na saída. Saída completa:\n{result.stdout}")
            continue

        # Extrai o JSON
        json_output = stdout[start_index:end_index + 1]
        try:
            simulation_result = json.loads(json_output)
        except json.JSONDecodeError:
            print(f"Tentativa {attempt + 1}: Falha ao decodificar JSON. JSON extraído:\n{json_output}")
            continue

        # Verifica se o status foi "ignored"
        if simulation_result.get("status") == "ignored":
            print(f"Tentativa {attempt + 1}: Simulação ignorada devido ao canal.")
            continue

        # Ajusta o índice da simulação no JSON
        simulation_result["simulation"] = simulation_index

        # Incrementa o índice da simulação para resultados válidos
        print(f"Tentativa {simulation_index}:")
        print(json.dumps(simulation_result, indent=4))  # Exibe o JSON atualizado com a contagem correta

        # Verifica o sucesso da simulação
        if simulation_result.get("success"):
            success_simulations += 1
            success_attempts.append(simulation_index)
        else:
            error_simulations += 1
            error_attempts.append(simulation_index)

        # Incrementa o índice da simulação após registrar
        simulation_index += 1

    # except subprocess.CalledProcessError as e:
        # print(f"Erro na execução da simulação {attempt + 1}: {e}")
        # continue
    except Exception as e:
        # print(f"Tentativa {attempt + 1}: Erro inesperado: {e}")
        continue

# Processa o arquivo CSV para acumular as métricas e renumerar as simulações
try:
    renumbered_rows = []
    with open(csv_file_path, mode='r') as file:
        reader = csv.reader(file)
        header = next(reader)  # Pula o cabeçalho

        # Identifica o índice das colunas, especialmente a nova coluna de erro
        try:
            timeslot_idx = header.index("Timeslot Total")
            eprs_created_idx = header.index("EPRs Criados")
            eprs_consumed_idx = header.index("Pares Eprs Consumidos")
            fidelity_avg_idx = header.index("Fidelidade Média dos Eprs")
            error_idx = header.index("Erro")
            sim_idx = header.index("Simulação")  # Para capturar o número da simulação
        except ValueError as ve:
            print(f"Erro ao localizar índices das colunas: {ve}")
            sys.exit(1)

        for row_num, row in enumerate(reader, start=1):
            # Verifica se a linha tem o número esperado de colunas
            if len(row) < len(header):
                print(f"Linha inválida no CSV na linha {row_num}: {row}")
                continue

            try:
                # Renumera as simulações linearmente
                row[sim_idx] = len(renumbered_rows) + 1
                renumbered_rows.append(row)

                # Atualiza as métricas acumuladas
                timeslot_total = int(row[timeslot_idx])
                eprs_created = int(row[eprs_created_idx])
                eprs_consumed = int(row[eprs_consumed_idx])
                fidelity_avg_raw = row[fidelity_avg_idx]
                erro = row[error_idx]

                # Deserializa a fidelidade média
                fidelity_avg = 0.0
                try:
                    fidelity_avg = json.loads(fidelity_avg_raw.replace("'", "\""))
                    if isinstance(fidelity_avg, dict) and 'media' in fidelity_avg:
                        fidelity_avg = fidelity_avg['media']
                    else:
                        fidelity_avg = float(fidelity_avg_raw)
                except json.JSONDecodeError:
                    try:
                        fidelity_avg = float(fidelity_avg_raw)
                    except ValueError:
                        fidelity_avg = 0.0

                # Atualiza as métricas acumuladas
                total_timeslots += timeslot_total
                total_eprs_created += eprs_created
                total_eprs_consumed += eprs_consumed
                if isinstance(fidelity_avg, (int, float)) and fidelity_avg > 0:
                    total_fidelity += fidelity_avg
                    fidelity_count += 1

            except (IndexError, ValueError) as e:
                # print(f"Erro ao processar a linha {row_num}: {row}. Erro: {e}")
                continue

    # Sobrescreve o CSV com as simulações renumeradas
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(renumbered_rows)

except FileNotFoundError:
    print(f"Arquivo {csv_file_path} não encontrado.")
except Exception as e:
    print(f"Erro ao processar o arquivo CSV: {e}")

# Calcula a fidelidade média total
average_fidelity = total_fidelity / fidelity_count if fidelity_count > 0 else 0.0

# Exibe o resumo das métricas no final
print("\nResumo das Métricas da Simulação (após todas as tentativas):")
print(f"Total de Timeslots: {total_timeslots}")
print(f"Total de EPRs Criados: {total_eprs_created}")
print(f"Total de EPRs Consumidos: {total_eprs_consumed}")
print(f"Fidelidade Média Total: {average_fidelity:.4f}")

# Exibe o resumo de simulações bem-sucedidas
print(f"A quantidade de simulações que deu certo é {success_simulations}")
print(f"As simulações que deram certo foram {success_attempts}")

# Exibe o resumo de simulações com erro
print(f"A quantidade de simulações que falharam é {error_simulations}")
print(f"As simulações que falharam foram {error_attempts}")


Tentativa 1:
{
    "simulation": 1,
    "success": false,
    "estimated_rounds": 2,
    "request": 0.85,
    "channel_info": "Y",
    "initial_fidelity": 0.7,
    "final_fidelity": 0.8448275862068965
}
Tentativa 2:
{
    "simulation": 2,
    "success": false,
    "estimated_rounds": 2,
    "request": 0.85,
    "channel_info": "X",
    "initial_fidelity": 0.7,
    "final_fidelity": 0.7
}
Tentativa 3:
{
    "simulation": 3,
    "success": true,
    "estimated_rounds": 2,
    "request": 0.85,
    "channel_info": "Y",
    "initial_fidelity": 0.7,
    "final_fidelity": 0.967365028203062
}
Tentativa 4:
{
    "simulation": 4,
    "success": false,
    "estimated_rounds": 2,
    "request": 0.85,
    "channel_info": "Y",
    "initial_fidelity": 0.7,
    "final_fidelity": 0.7
}
Tentativa 5:
{
    "simulation": 5,
    "success": false,
    "estimated_rounds": 2,
    "request": 0.85,
    "channel_info": "X",
    "initial_fidelity": 0.7,
    "final_fidelity": 0.7
}
Tentativa 6:
{
    "simulation":